In [ ]:
def is_colab():
    try:
        from google.colab import drive
        return True
    except:
        return False

In [ ]:
if is_colab():
    !git clone https://github.com/Priyanshu-hawk/pytorch_random.git

Cloning into 'pytorch_random'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 72 (delta 25), reused 53 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 4.38 MiB | 9.43 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
import os
if is_colab() and os.getcwd() != '/content/pytorch_random': #vauge but works
    %cd /content/pytorch_random

/content/pytorch_random


In [ ]:
!pip install -r req.txt
!pip install requests

In [ ]:
import requests
with open("./Modular_DeepLearning/food101_3_class/data.zip", "wb") as f:
  r = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip")
  f.write(r.content)

import zipfile, os
with zipfile.ZipFile("./Modular_DeepLearning/food101_3_class/data.zip", "r") as zip_ref:
    os.makedirs("./Modular_DeepLearning/food101_3_class/data", exist_ok=True)
    zip_ref.extractall("./Modular_DeepLearning/food101_3_class/data")

In [ ]:
import os
data = "./Modular_DeepLearning/food101_3_class/data"
train_path = os.path.join(data,"train")
test_path = os.path.join(data,"test")

In [ ]:
from Modular_DeepLearning.food101_3_class.data_setup import createDataloader
import torchvision, torch

In [ ]:
# model_weights = torchvision.models.ResNet101_Weights.DEFAULT
# _model = torchvision.models.resnet101(pretrained=model_weights)

model_weights = torchvision.models.RegNet_X_32GF_Weights.IMAGENET1K_V2
_model = torchvision.models.regnet_x_32gf(weights=model_weights)

auto_transform = model_weights.transforms()

auto_transform

Downloading: "https://download.pytorch.org/models/regnet_x_32gf-6eb8fdc6.pth" to /root/.cache/torch/hub/checkpoints/regnet_x_32gf-6eb8fdc6.pth
100%|██████████| 412M/412M [00:04<00:00, 91.3MB/s]


ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
train_load, test_load, class_names = createDataloader(train_path, test_path, BTACH_SIZE=32, train_transform=auto_transform, test_transform=auto_transform)


In [ ]:
from torchinfo import summary
summary(_model, input_size=(1, 3, 224, 224),
        col_names=("input_size", "output_size", "num_params", "trainable"),
        row_settings=["var_names"])

Layer (type (var_name))                            Input Shape               Output Shape              Param #                   Trainable
RegNet (RegNet)                                    [1, 3, 224, 224]          [1, 1000]                 --                        Partial
├─SimpleStemIN (stem)                              [1, 3, 224, 224]          [1, 32, 112, 112]         --                        False
│    └─Conv2d (0)                                  [1, 3, 224, 224]          [1, 32, 112, 112]         (864)                     False
│    └─BatchNorm2d (1)                             [1, 32, 112, 112]         [1, 32, 112, 112]         (64)                      False
│    └─ReLU (2)                                    [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
├─Sequential (trunk_output)                        [1, 32, 112, 112]         [1, 2520, 7, 7]           --                        False
│    └─AnyStage (block1)                           [

In [ ]:
#pause grading for feature layer
for prams in _model.stem.parameters():
    prams.requires_grad = False

for prams in _model.trunk_output.parameters():
    prams.requires_grad = False

In [ ]:
import torch
_model.fc = torch.nn.Linear(in_features=2520, out_features=len(class_names), bias=True)

In [ ]:
from torchinfo import summary
summary(_model, input_size=(1, 3, 224, 224),
        col_names=("input_size", "output_size", "num_params", "trainable"),
        row_settings=["var_names"])

Layer (type (var_name))                            Input Shape               Output Shape              Param #                   Trainable
RegNet (RegNet)                                    [1, 3, 224, 224]          [1, 3]                    --                        Partial
├─SimpleStemIN (stem)                              [1, 3, 224, 224]          [1, 32, 112, 112]         --                        False
│    └─Conv2d (0)                                  [1, 3, 224, 224]          [1, 32, 112, 112]         (864)                     False
│    └─BatchNorm2d (1)                             [1, 32, 112, 112]         [1, 32, 112, 112]         (64)                      False
│    └─ReLU (2)                                    [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
├─Sequential (trunk_output)                        [1, 32, 112, 112]         [1, 2520, 7, 7]           --                        False
│    └─AnyStage (block1)                           [

In [ ]:
_model = _model.to("cuda")

In [ ]:
from Modular_DeepLearning.food101_3_class.train import trainit

In [ ]:
loss_fn  = torch.nn.CrossEntropyLoss().to("cuda")
optimizer = torch.optim.Adam(_model.parameters(), lr=0.001)